# taken from my notes

In [ ]:
# very grim and not at all surprising that column with the most missings is IPV
# (from the ACE survey - so IPV among parents, not participants themselves)
# >>> meyer['w1ace_ipv'].value_counts(dropna = False)
# w1ace_ipv
# 0    950
# 1    429
#      139
# Name: count, dtype: int64
# >>> meyer['w1ace_ipv_i'].value_counts(dropna = False)
# w1ace_ipv_i
# 0    1024
# 1     494
# Name: count, dtype: int64
# >>> 1024-950
# 74
# >>> 139-74
# 65

# ok.  their imputation method seems really solid, but that is sus af.
# of the people who declined to answer about IPV, you think the majority 
# of them would have said NO!?!??!??????
# I'd be tempted to put every one of those guys down as a yes!!!!
# let me see the overlap with other stigmatized stuff
len(meyer[(meyer['w1ace_ipv']==' ') & (meyer['w1ace_sex']==' ')])
# 30 huh damn I would have thought like all of them
# wait.
len(meyer[(meyer['w1ace_ipv']=='1') & (meyer['w1ace_sex']==' ')])  # 19
len(meyer[(meyer['w1ace_ipv']==' ') & (meyer['w1ace_sex']=='1')])  # 51
# ok! that actually lends credibility to their imputation though
# if people were too ashamed to admit IPV, they'd probably decline to answer
# about CSA too.  these people, at least, really could go either way on IPV.

In [ ]:
# This is all actually moot 
# because I don't have a better idea for imputation than what they did.
# I think the thing to do, if I can, is to model it on the imputed data
# then model it again on the subsample where I've dropped all these NAs
# at least the ACE NAs, where there's really a lot.
# could also try modeling it just without those columns
# a random forest would probably be good, 
# make sure the whole thing isn't hinging on imputed data
# Meyer also probably ran some tests in his paper to make sure this is ok


# I've gotta reduce my dimensions
# the imputation methods they used seem very reasonable
# (see p. 37 of 37166-Documentation-methodology.pdf)
# so I'm going to drop the non-imputed ones

In [ ]:
# hey, is it just a few rows with a ton of missing data?
# I hate to lose data but if they truly didn't fill out anything...
meyer.iloc[10,:].isna().sum()
# oh right, there are all those OHE columns with planned missing
# so I need to start imputing first, then recheck these
# then decide whether to drop anything

# i did a whole clever investigation and i want to brag about it
but it's so much faster to just add this column to s_mode (so that's what I did)

In [ ]:
# subset investigation; there were exactly 48 do not knows on q2

cond1 = meyer['w1q03'].isna()

test = meyer.loc[cond1, ['w1q02_ei', 'w1q03']]
test['w1q02_ei'].value_counts(dropna = False, normalize = True) # why so many 8s?
meyer['w1q02_ei'].value_counts(dropna = False, normalize = True)  # close enough for now

# I conducted that test to see if the 48 NAs in Q3 were the same
# people as the 48 "don't knows" in  Q2, or any other pattern.
# I did not find one, and therefore imputed the mode, which is 2.

meyer[['w1q03_ei']] = meyer[['w1q03']].fillna(2)

meyer.shape
meyer.drop(columns = 'w1q03', inplace = True)

# taken from the draft of 02_Data_Preparation

# **MAYBE TAKE THIS OUT???  I CAN'T TELL IF IT'S CONVINCING**  vv

For a quick-and-dirty test of the facial validity of these imputations, I compared the value counts of `w1ace_ipv` (before imputation) and `w1ace_ipv_i` (after imputation).  This variable corresponds to questions from the Adverse Childhood Experiences (ACE) scale regarding intimate partner violence (IPV) in their home when they were a child.  IPV is notoriously underreported, and not surprisingly, this variable had one of the highest rates of missing data.

In [16]:
print(meyer['w1ace_ipv'].value_counts(dropna = False))
print(meyer['w1ace_ipv_i'].value_counts(dropna = False))

w1ace_ipv
0.0    945
1.0    426
NaN    136
Name: count, dtype: int64
w1ace_ipv_i
0    1018
1     489
Name: count, dtype: int64


In [17]:
1010-940, 484-424

(70, 60)

In [18]:
len(meyer[(meyer['w1ace_ipv']==1) & (meyer['w1ace_sex'].isna())])

19

In [19]:
len(meyer[(meyer['w1ace_ipv'].isna()) & (meyer['w1ace_sex']==1)])

50

In [20]:
meyer[(meyer['w1ace_ipv'].isna()) & (meyer['w1ace_sex']==1)]['w1ace_ipv_i'].value_counts(dropna = False)

w1ace_ipv_i
1    27
0    23
Name: count, dtype: int64

In [21]:
meyer[(meyer['w1ace_ipv'].isna()) & (meyer['w1ace_ipv_i']==0)]['w1ace_sex'].value_counts(dropna = False, normalize = True)

w1ace_sex
0.0    0.438356
1.0    0.315068
NaN    0.246575
Name: proportion, dtype: float64

In [22]:
meyer[(meyer['w1ace_ipv'].isna()) & (meyer['w1ace_ipv_i']==1)]['w1ace_sex'].value_counts(dropna = False, normalize = True)

w1ace_sex
1.0    0.428571
0.0    0.380952
NaN    0.190476
Name: proportion, dtype: float64

In [23]:
meyer['w1ace_ipv'].value_counts(dropna = True, normalize = True)

w1ace_ipv
0.0    0.689278
1.0    0.310722
Name: proportion, dtype: float64

In [24]:
len(meyer[(meyer['w1ace_ipv']==' ') & (meyer['w1ace_sex']=='1')])

0

Of the 130 missing values in the original column, 

# ^^ **MAYBE TAKE THIS OUT???  I CAN'T TELL IF IT'S CONVINCING**

Notably, I chose to impute these values before completing a train test split.  This is partially because I am primarily interested in an inferential model, rather than a predictive one.  Given how few observations I have in the dataset, I decided to prioritize fidelity to that data over adaptability to new data in the future.  Perhaps more importantly though, because this data was collected at a particular time and place in history, and addresses sensitive, often fraught, personal topics, there is no reason to expect that any model of it should predict future data, collected at a different time and place in history, as well as it explains this immediate data.  Unlike, for instance, sales of a product, queer people's understandings of themselves and their identities, and their experiences of community, homophobia, discrimination, etc., are very likely to change over time. Previous research has established that the same features tend to be consistenly *meaningful* predictors of queer mental health (e.g., Meyer has been publishing papers on minority stress in queer people since at least [1995](https://www.jstor.org/stable/2137286?origin=JSTOR-pdf), and the concept goes back decades further), but the exact weights and interplays of different factors (i.e., beta coefficients) are likely to vary with the cultural ups and downs.  Even a perfect model, fit on data from 2016, is likely to perform poorly when applied to more recent data.  In short, it would be more useful for future researchers to construct their own models and compare them to mine than it would be to apply my model to new data; therefore, I opted to maximize the information available for my imputers, rather than sacrificing any of it for a ill-conceived hope of future generalizability.